# 03 - Battery Stress Detection Model Training

This notebook trains a neural network model to detect battery stress conditions. The model learns to identify when a battery is under stress based on parameters like current, temperature, voltage, and state of charge.


## Install OpenVINO Development Tools

Install OpenVINO toolkit for model optimization and conversion to Intel's Intermediate Representation (IR) format.


In [ ]:
!pip install openvino

## Disable GPU Usage

Configure TensorFlow to use CPU only, ensuring consistent execution across different environments.


In [ ]:
import os
import warnings

os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

warnings.filterwarnings('ignore', category=UserWarning, module='google.protobuf')
warnings.filterwarnings('ignore', category=DeprecationWarning)

## Import Required Libraries

Import all necessary libraries for data processing, model building, and training.


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
import tensorflow as tf
from tensorflow import keras
import joblib
import os
import subprocess
import shutil
import json

## Load Prepared Battery Data

Read the prepared battery data from the CSV file created in the previous notebook.


In [ ]:
df = pd.read_csv("./data/battery_data.csv")

## Create Stress Indicator Labels

Define what constitutes a "stressed" battery based on threshold values for current, temperature, state of charge, and voltage. Create binary labels (0=normal, 1=stressed) for model training.


In [ ]:
def detect_stress(row):
    if row["batteryCurrent"] > 400 or row["batteryTemp"] > 50 or row["stateOfCharge"] < 0.05 or row["batteryVoltage"] < 320:
        return 1
    return 0

df["stressIndicator"] = df.apply(detect_stress, axis=1)

## Select Features and Target Variable

Choose the input features (sensor readings) and the target variable (stress indicator) for the model.


In [ ]:
features = ["stateOfCharge", "stateOfHealth", "batteryCurrent", "batteryVoltage", "kmh", "distance", "batteryTemp", "ambientTemp", "currentLoad"]
X = df[features]
y = df["stressIndicator"]

## Split and Normalize Data

Split the data into training and testing sets, then normalize the features using StandardScaler to improve model performance.


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

X_train_scaled = np.asarray(X_train_scaled, dtype=np.float32)
X_test_scaled = np.asarray(X_test_scaled, dtype=np.float32)
y_train = np.asarray(y_train, dtype=np.float32)
y_test = np.asarray(y_test, dtype=np.float32)

## Build Neural Network Model

Create a Multi-Layer Perceptron (MLP) with two hidden layers for binary classification of battery stress.


In [ ]:
mlp_tf = keras.Sequential([
    keras.layers.Input(shape=(X_train.shape[1],)),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

mlp_tf.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

## Train the Model

Train the neural network on the battery data for 50 epochs, using 10% of training data for validation.


In [ ]:
mlp_tf.fit(X_train_scaled, y_train, epochs=50, batch_size=32, validation_split=0.1)

## Save Model and Scaler

Save the trained model and the data scaler for later use in prediction.


In [ ]:
os.makedirs("models", exist_ok=True)
scaler_path = "models/stress_scaler.pkl"
joblib.dump(scaler, scaler_path)
mlp_tf.save("models/battery_stress_model.keras")

## Export Model to TensorFlow SavedModel Format

Export the model in TensorFlow's SavedModel format, which is needed for conversion to OpenVINO IR.


In [ ]:
try:
    mlp_tf.export("models/battery_stress_model")
except AttributeError:
    mlp_tf.save("models/battery_stress_model", save_format='tf')

## Convert Model to OpenVINO IR Format

Convert the TensorFlow model to OpenVINO Intermediate Representation (IR) for optimized inference on Intel hardware.


In [ ]:
os.makedirs("models/battery_stress_model_ir", exist_ok=True)

subprocess.run(
    [
        "ovc",
        "models/battery_stress_model",
        "--output_model", "models/battery_stress_model_ir/saved_model"
    ],
    capture_output=True,
    text=True,
    check=True
)

## Prepare Model for Serving

Organize the IR model files into a directory structure suitable for serving with OpenVINO Model Server.


In [ ]:

def prepare_ir_model_for_serving(ir_model_path, output_path, version=1):
    version_path = os.path.join(output_path, str(version))
    os.makedirs(version_path, exist_ok=True)
    
    for item in os.listdir(ir_model_path):
        if item.endswith(('.xml', '.bin')):
            src = os.path.join(ir_model_path, item)
            dst = os.path.join(version_path, item)
            shutil.copy2(src, dst)
    
    return version_path

serving_path = "models/serving/stress-detection"
ir_model_path = "models/battery_stress_model_ir"

model_version_path = prepare_ir_model_for_serving(ir_model_path, serving_path, version=1)